In [ ]:
#set up plotting style 
sns.set_theme()

# Create a figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Popularity distribution
sns.histplot(data=df, x='popularity', kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Distribution of Track Popularity')
axes[0, 0].set_xlabel('Popularity')

# 2. Top 10 artists by track count
top_artists = df['artist'].value_counts().head(10)
sns.barplot(x=top_artists.values, y=top_artists.index, ax=axes[0, 1])
axes[0, 1].set_title('Top 10 Artists by Track Count')
axes[0, 1].set_xlabel('Number of Tracks')

# 3. Correlation between popularity and other numeric features
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlations = df[numeric_cols].corr()['popularity'].sort_values(ascending=False)
sns.barplot(x=correlations.values[1:], y=correlations.index[1:], ax=axes[1, 1])
axes[1, 1].set_title('Correlation with Popularity')
axes[1, 1].set_xlabel('Correlation Coefficient')

plt.tight_layout()
plt.show()